### Import Libraries

In [ ]:
import string
from typing import List, Dict
from typing_extensions import Annotated
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# LangChain Core
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_core.prompts import (
    PromptTemplate,
    FewShotPromptTemplate,
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.exceptions import OutputParserException
from langchain_core.runnables import (
    RunnableSequence,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

# LangChain Output Parsers
from langchain.output_parsers.datetime import DatetimeOutputParser
from langchain.output_parsers.boolean import BooleanOutputParser
from langchain.output_parsers import OutputFixingParser

# LangChain RAG Components
from langchain_community.document_loaders.arxiv import ArxivLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore

# LangChain Tools
from langchain.tools import tool
from langchain_core.tools.structured import StructuredTool
from langchain_core.output_parsers.openai_tools import parse_tool_calls

from dotenv import load_dotenv

### Bascis

#### Simple LLM Call

In [ ]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)
print(llm.invoke("What is Alphafold?"))

content="AlphaFold is an artificial intelligence program developed by DeepMind, a subsidiary of Alphabet Inc. It is designed to predict the three-dimensional structures of proteins based on their amino acid sequences. Proteins are essential biological molecules that perform a wide range of functions in living organisms, and their function is closely related to their structure. \n\nAlphaFold gained significant attention for its performance in the Critical Assessment of protein Structure Prediction (CASP) competition, where it demonstrated remarkable accuracy in predicting protein structures, often rivaling experimental methods such as X-ray crystallography and cryo-electron microscopy. The program uses deep learning techniques to analyze patterns in known protein structures and make predictions about new ones.\n\nThe implications of AlphaFold's capabilities are vast, potentially accelerating research in fields such as drug discovery, disease understanding, and synthetic biology by provi

#### Chat Structure

In [ ]:
messages = [
    SystemMessage("You are a Data Scientist in biotech"),
    HumanMessage("What is AlphaFold used for?"),
]
ai_message = llm.invoke(messages)
print(ai_message.content)

AlphaFold is an artificial intelligence program developed by DeepMind that predicts protein structures with high accuracy. It uses deep learning techniques to model the three-dimensional shapes of proteins based on their amino acid sequences. The primary applications of AlphaFold include:

1. **Understanding Protein Function**: By predicting the structure of proteins, researchers can gain insights into their functions and how they interact with other molecules, which is crucial for understanding biological processes.

2. **Drug Discovery**: Accurate protein structures can aid in the design of new drugs by allowing scientists to identify potential binding sites and optimize drug candidates for better efficacy and reduced side effects.

3. **Disease Research**: AlphaFold can help in understanding the structural basis of diseases caused by protein misfolding or mutations, such as Alzheimer's disease or cystic fibrosis.

4. **Synthetic Biology**: Researchers can use AlphaFold to design new

In [ ]:
messages = [
    SystemMessage("You are a Data Scientist in biotech"),
    HumanMessage("What is AlphaFold used for?"),
    AIMessage("AlphaFold is used to predict the structure of proteins."),
    HumanMessage("What is RFdiffusion used for?"),
]

ai_message = llm.invoke(messages)
print(ai_message.content)

RFdiffusion is a generative model used for protein design and structure prediction. It leverages diffusion models to generate novel protein sequences and structures by learning from existing protein data. RFdiffusion can help in tasks such as designing proteins with specific functions, optimizing protein stability, and exploring the vast sequence space of proteins to identify potential candidates for therapeutic or industrial applications. Its ability to generate high-quality protein structures makes it a valuable tool in the fields of biotechnology and drug discovery.


#### Prompt Templates

In [ ]:
prompt_template = PromptTemplate(
    template="What is {tool} used for?",
)

print(prompt_template.invoke({"tool": "Alphafold"}))

text='What is Alphafold used for?'


In [ ]:
ai_message = llm.invoke(prompt_template.invoke({"tool": "Alphafold"}))
print(ai_message.content)

AlphaFold is an artificial intelligence program developed by DeepMind that predicts protein structures with high accuracy. It is primarily used for:

1. **Protein Structure Prediction**: AlphaFold can predict the three-dimensional structures of proteins based on their amino acid sequences. This is crucial for understanding how proteins function and interact with other molecules.

2. **Biological Research**: Researchers use AlphaFold to gain insights into various biological processes, including enzyme function, protein-protein interactions, and the mechanisms of diseases.

3. **Drug Discovery**: By providing accurate protein structures, AlphaFold aids in the design of new drugs and therapeutic interventions by helping scientists understand the target proteins involved in diseases.

4. **Genomics and Proteomics**: AlphaFold can assist in interpreting genomic data by predicting the structures of proteins encoded by newly sequenced genes, which can lead to discoveries in functional genomic

#### Few Shot Prompt

In [ ]:
examples = [
    {
        "input": "A bioreactor is seeded with 2×10⁶ cells/mL. The cells double every 24 hours. What will the cell density be after 3 days?",
        "thought": "The cells double once every 24 hours. In 3 days (72 hours), there are 3 doublings: 2×10⁶ × 2³ = 2×10⁶ × 8 = 1.6×10⁷ cells/mL.",
        "output": "1.6×10⁷ cells/mL",
    },
    {
        "input": "A downstream purification step has a yield of 80%. If the initial protein amount is 5 grams, how much protein remains after this step?",
        "thought": "The yield is 80%, so the remaining protein is 5 g × 0.8 = 4 g.",
        "output": "4 grams",
    },
    {
        "input": "A fermentation process uses glucose and produces ethanol and CO₂. If 10 moles of glucose are consumed, how many moles of ethanol are produced, assuming the theoretical yield (2 moles ethanol per mole glucose)?",
        "thought": "The stoichiometry is 1 glucose → 2 ethanol. Thus, 10 moles glucose × 2 = 20 moles ethanol.",
        "output": "20 moles ethanol",
    },
    {
        "input": "A cell culture produces 150 mg of antibody in 5 liters over 48 hours. What is the average production rate in mg/L/hour?",
        "thought": "Production rate = total amount / (volume × time) = 150 mg / (5 L × 48 h) = 150 / 240 = 0.625 mg/L/h.",
        "output": "0.625 mg/L/h",
    },
    {
        "input": "A fed-batch process starts with 1 liter at 10 g/L substrate. 2 liters of feed at 20 g/L are added. What is the final substrate concentration (assuming perfect mixing and no consumption)?",
        "thought": "Initial substrate: 1 L × 10 g/L = 10 g. Feed: 2 L × 20 g/L = 40 g. Total substrate = 10 + 40 = 50 g. Final volume = 1 + 2 = 3 L. Final concentration = 50 g / 3 L ≈ 16.67 g/L.",
        "output": "16.67 g/L",
    },
]


example_prompt = PromptTemplate(
    template="Question: {input}\nThought: {thought}\nResponse: {output}"
)

print(example_prompt.invoke(examples[0]).to_string())

Question: A bioreactor is seeded with 2×10⁶ cells/mL. The cells double every 24 hours. What will the cell density be after 3 days?
Thought: The cells double once every 24 hours. In 3 days (72 hours), there are 3 doublings: 2×10⁶ × 2³ = 2×10⁶ × 8 = 1.6×10⁷ cells/mL.
Response: 1.6×10⁷ cells/mL


In [ ]:
prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

response = llm.invoke(
    prompt_template.invoke(
        {
            "input": "What is the final concentration of a solution that starts with 50 mL of 0.1 M NaCl and is diluted to 200 mL?"
        }
    )
)
print(response.content)  # Response should be 0.025 M NaCl

To find the final concentration after dilution, we can use the dilution formula:

\[ C_1V_1 = C_2V_2 \]

Where:
- \( C_1 \) is the initial concentration (0.1 M)
- \( V_1 \) is the initial volume (50 mL)
- \( C_2 \) is the final concentration (unknown)
- \( V_2 \) is the final volume (200 mL)

Rearranging the formula to solve for \( C_2 \):

\[ C_2 = \frac{C_1V_1}{V_2} \]

Substituting the known values:

\[ C_2 = \frac{0.1 \, \text{M} \times 50 \, \text{mL}}{200 \, \text{mL}} \]

Calculating:

\[ C_2 = \frac{5 \, \text{mmol}}{200 \, \text{mL}} = 0.025 \, \text{M} \]

Thus, the final concentration of the solution is:

**0.025 M NaCl**


### Streaming

In [ ]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

#### Chunking

In [ ]:
def play(message: str, memory: List):
    memory.append(HumanMessage(content=message))
    chunks = []
    try:
        for chunk in llm.stream(memory):
            chunks.append(chunk)
            print(chunk.content, end="|", flush=True)
            if len(chunks) % 12 == 0:
                print("\n")
    except KeyboardInterrupt:
        print("\n______________________________")

    result = "".join([chunk.content for chunk in chunks])
    memory.append(AIMessage(content=result))


def resume(memory: List):
    print("\nResuming from last interaction...\n")
    play(
        message="If your last message is not complete, continue "
        "after the last word. If it's complete, just output __END__",
        memory=memory,
    )

In [35]:
memory = []

message = "What is the final concentration of a solution that starts with 50 mL of 0.1 M NaCl and is diluted to 200 mL?"

play(message, memory)

|To| find| the| final| concentration| of| the| solution| after| dilution|,|

 you| can| use| the| dilution| formula|:

|\|[
|C|_|1|

 V|_|1| =| C|_|2| V|_|2|
|\|

]

|where|:
|-| \(|C|_|1|\|)| is| the|

 initial| concentration| (|0|.|1| M|),
|-| \(|V|_|

1|\|)| is| the| initial| volume| (|50| m|L|),
|

-| \(|C|_|2|\|)| is| the| final| concentration| (|

what| we| want| to| find|),
|-| \(|V|_|2|\|

)| is| the| final| volume| (|200| m|L|).

|Plug|ging|

 in| the| values|:

|\|[
|(|0|.|1| \|,|

 \|text|{|M|})| \|times| (|50| \|,| \|

text|{|m|L|})| =| C|_|2| \|times| (|

200| \|,| \|text|{|m|L|})
|\|]

|Calcul|

ating| the| left| side|:

|\|[
|5| \|,| \|text|

{|M| m|L|}| =| C|_|2| \|times| |

200| \|,| \|text|{|m|L|}
|\|]

|Now|

,| solve| for| \(|C|_|2|\|):

|\|[
|C|

_|2| =| \|frac|{|5| \|,| \|text|{|

M| m|L|}}|{|200| \|,| \|text|{|m|

L|}}| =| |0|.|025| \|,| \|text|{|

M|}
|\|]

|Thus|,| the| final| concentration| of| the| solution|

 after| dilution| is| **|0|.|025| M|**|.||

In [36]:
resume(memory)


Resuming from last interaction...

|__|END|__||

In [37]:
memory

[HumanMessage(content='What is the final concentration of a solution that starts with 50 mL of 0.1 M NaCl and is diluted to 200 mL?'),
 AIMessage(content='To find the final concentration of the solution after dilution, you can use the dilution formula:\n\n\\[\nC_1 V_1 = C_2 V_2\n\\]\n\nwhere:\n- \\(C_1\\) is the initial concentration (0.1 M),\n- \\(V_1\\) is the initial volume (50 mL),\n- \\(C_2\\) is the final concentration (what we want to find),\n- \\(V_2\\) is the final volume (200 mL).\n\nPlugging in the values:\n\n\\[\n(0.1 \\, \\text{M}) \\times (50 \\, \\text{mL}) = C_2 \\times (200 \\, \\text{mL})\n\\]\n\nCalculating the left side:\n\n\\[\n5 \\, \\text{M mL} = C_2 \\times 200 \\, \\text{mL}\n\\]\n\nNow, solve for \\(C_2\\):\n\n\\[\nC_2 = \\frac{5 \\, \\text{M mL}}{200 \\, \\text{mL}} = 0.025 \\, \\text{M}\n\\]\n\nThus, the final concentration of the solution after dilution is **0.025 M**.'),
 HumanMessage(content="If your last message is not complete, continue after the last w

#### Streaming Events

In [38]:
events = []
async for event in llm.astream_events("hello", version="v2"):
    if event["event"] == "on_chat_model_start":
        print("Streaming...")
    if event["event"] == "on_chat_model_stream":
        print(
            f"Chat model chunk: {repr(event['data']['chunk'].content)}",
            flush=True,
        )
        events.append(event)
    if event["event"] == "on_chat_model_end":
        # It could trigger another process
        print("__END__")

/var/folders/f3/_6ts5bqd5s30t0mq76kjkdxc0000gn/T/ipykernel_3992/3603337307.py:2: LangChainBetaWarning: This API is in beta and may change in the future.
  async for event in llm.astream_events("hello", version="v2"):


Streaming...
Chat model chunk: ''
Chat model chunk: 'Hello'
Chat model chunk: '!'
Chat model chunk: ' How'
Chat model chunk: ' can'
Chat model chunk: ' I'
Chat model chunk: ' assist'
Chat model chunk: ' you'
Chat model chunk: ' today'
Chat model chunk: '?'
Chat model chunk: ''
__END__


In [ ]:
class ChatBot:
    def __init__(
        self,
        name: str,
        instructions: str,
        examples: List[dict],
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
    ):

        self.llm = ChatOpenAI(
            model=model,
            temperature=temperature,
        )

        system_prompt = SystemMessage(instructions)
        example_prompt = ChatPromptTemplate.from_messages(
            [
                ("human", "{input}"),
                ("ai", "{output}"),
            ]
        )
        prompt_template = FewShotChatMessagePromptTemplate(
            example_prompt=example_prompt,
            examples=examples,
        )

        self.messages = prompt_template.invoke({}).to_messages()

    async def invoke(self, user_message: str) -> AIMessage:
        self.messages.append(HumanMessage(user_message))
        events = []
        chunks = []

        # Replacing invoke()
        async for event in llm.astream_events(self.messages, version="v2"):
            events.append(event)
            if event["event"] == "on_chat_model_start":
                print("Streaming...")
            if event["event"] == "on_chat_model_stream":
                chunk = event["data"]["chunk"]
                chunks.append(chunk)
                print(chunk.content, end="", flush=True)
                if chunk.content.strip() in string.punctuation:
                    print("\n")

            if event["event"] == "on_chat_model_end":
                ai_message = AIMessage(event["data"]["output"].content)
                self.messages.append(ai_message)

In [ ]:
instructions = (
    "Your task is to answer questions about the given text. "
    "Answer as concisely as possible. "
    "Use the following format: "
    "Question: [question] "
    "Answer: [answer]"
)

examples = [
    {
        "input": "What is the capital of France?",
        "output": "The capital of France is Paris.",
    },
    {
        "input": "What is the largest planet in our solar system?",
        "output": "The largest planet in our solar system is Jupiter.",
    },
    {
        "input": "What is the chemical symbol for gold?",
        "output": "The chemical symbol for gold is Au.",
    },
]

chatbot = ChatBot(
    name="ChatGPT",
    instructions=instructions,
    examples=examples,
)

await chatbot.invoke("What is the chemical symbol for Silver?")

Streaming...


The chemical symbol for silver is Ag.





### Output Parsers

In [43]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

#### String

In [44]:
llm.invoke("hello").content

'Hello! How can I assist you today?'

In [ ]:
parser = StrOutputParser()

parser.invoke(llm.invoke("hello"))

'Hello! How can I assist you today?'

#### Datetime

In [ ]:
llm.invoke(
    "Output a random datetime in %Y-%m-%dT%H:%M:%S.%fZ. " "Don't say anything else"
)

AIMessage(content='2023-10-05T14:23:45.123456Z', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 33, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'stop', 'logprobs': None}, id='run-635b06b1-bdb6-44d1-9728-8c3e215515bd-0', usage_metadata={'input_tokens': 33, 'output_tokens': 17, 'total_tokens': 50})

In [ ]:
parser = DatetimeOutputParser()

parser.invoke(
    llm.invoke(
        "Output a random datetime in %Y-%m-%dT%H:%M:%S.%fZ. " "Don't say anything else"
    )
)

datetime.datetime(2023, 10, 5, 14, 23, 45, 123456)

#### Boolean

In [ ]:
llm.invoke("Are you an AI? YES or NO only")

AIMessage(content='YES', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 16, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'finish_reason': 'stop', 'logprobs': None}, id='run-0e3fcc28-484a-40f8-9e6d-9ba05eeba6c6-0', usage_metadata={'input_tokens': 16, 'output_tokens': 2, 'total_tokens': 18})

In [ ]:
parser = BooleanOutputParser()

parser.invoke(input=llm.invoke("Are you an AI?"))

True

#### Structured

In [ ]:
class PydanticModelInfo(BaseModel):
    model_name: Annotated[
        str,
        Field(
            description="Paper referred model abbreviation names. Defaults to ''",
            default=None,
        ),
    ]
    publish_year: Annotated[
        str,
        Field(description="When was this model proposed. Defaults to ''", default=None),
    ]
    model_type: Annotated[
        str,
        Field(
            description="Type of model (e.g., discriminative, generative)", default=None
        ),
    ]
    application_domain: Annotated[
        str,
        Field(
            description="Domain where the model is applied (e.g., antibody design)",
            default=None,
        ),
    ]
    authors: Annotated[
        str, Field(description="Authors of the paper (e.g., Luo et al.)", default=None)
    ]
    model_category: Annotated[
        str,
        Field(
            description="Specific category of model (e.g., diffusion model, graph neural network)",
            default=None,
        ),
    ]
    related_models: Annotated[
        list[str],
        Field(
            description="Other models mentioned in the same context",
            default_factory=list,
        ),
    ]
    description: Annotated[
        str,
        Field(
            description="Brief description of the model's purpose or approach",
            default=None,
        ),
    ]

In [ ]:
llm_with_structure = llm.with_structured_output(PydanticModelInfo)

structured_output = llm_with_structure.invoke(
    """
    Recent deep learning-based methods for antibody design can be broadly categorized into discrimina-
    tive models and generative models. Discriminative models typically leverage graph neural networks
    to learn the presentations of antigen structure and to predict the most likely antibody structure and
    sequence (Kong et al., 2023; Lin et al., 2024). In contrast, generative models such as the denoising
    diffusion probabilistic models (DDPM) (Luo et al., 2022; Martinkus et al., 2024; Tan et al., 2024)
    and score-based diffusion models (Zhu et al., 2024; Kulyt˙ e et al., 2024) build an antigen-conditioned
    generative process of antibody sequences and structure.
    """
)
print(structured_output)

model_name='Denoising Diffusion Probabilistic Models' publish_year='2022' model_type='generative' application_domain='antibody design' authors='Luo et al.' model_category='diffusion model' related_models=[] description='Generative models that build an antigen-conditioned generative process of antibody sequences and structure.'


####  Dealing with Errors

In [54]:
misformatted_result = "{'model_name': 'DDPM', 'publish_year': '2022', 'model_type': 'generative', 'application_domain': 'antibody design', 'authors': 'Luo et al.', 'model_category': 'diffusion model', 'related_models': ['DDPM', 'DDPM', 'DDPM'], 'description': 'Denoising diffusion probabilistic models (DDPM) are a type of generative model that learns to generate new data by iteratively denoising a noisy input. In the context of antibody design, DDPMs can be used to generate new antibody sequences and structures that are similar to a given antigen.'}"
print(misformatted_result)

{'model_name': 'DDPM', 'publish_year': '2022', 'model_type': 'generative', 'application_domain': 'antibody design', 'authors': 'Luo et al.', 'model_category': 'diffusion model', 'related_models': ['DDPM', 'DDPM', 'DDPM'], 'description': 'Denoising diffusion probabilistic models (DDPM) are a type of generative model that learns to generate new data by iteratively denoising a noisy input. In the context of antibody design, DDPMs can be used to generate new antibody sequences and structures that are similar to a given antigen.'}


In [56]:
parser = PydanticOutputParser(pydantic_object=PydanticModelInfo)

try:
    parser.parse(misformatted_result)
except OutputParserException as e:
    print(e)

Invalid json output: {'model_name': 'DDPM', 'publish_year': '2022', 'model_type': 'generative', 'application_domain': 'antibody design', 'authors': 'Luo et al.', 'model_category': 'diffusion model', 'related_models': ['DDPM', 'DDPM', 'DDPM'], 'description': 'Denoising diffusion probabilistic models (DDPM) are a type of generative model that learns to generate new data by iteratively denoising a noisy input. In the context of antibody design, DDPMs can be used to generate new antibody sequences and structures that are similar to a given antigen.'}


In [57]:
new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

new_parser.parse(misformatted_result)

PydanticModelInfo(model_name='DDPM', publish_year='2022', model_type='generative', application_domain='antibody design', authors='Luo et al.', model_category='diffusion model', related_models=['DDPM', 'DDPM', 'DDPM'], description='Denoising diffusion probabilistic models (DDPM) are a type of generative model that learns to generate new data by iteratively denoising a noisy input. In the context of antibody design, DDPMs can be used to generate new antibody sequences and structures that are similar to a given antigen.')

### Lang Chain Expression Language

#### Chaining invocations

In [ ]:
load_dotenv()

prompt = PromptTemplate(
    template="What is {tool} used for?",
)


llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

parser = StrOutputParser()

In [ ]:
parser.invoke(llm.invoke(prompt.invoke({"tool": "Alphafold"})))

'AlphaFold is an artificial intelligence program developed by DeepMind that predicts protein structures with high accuracy. It is primarily used for:\n\n1. **Protein Structure Prediction**: AlphaFold can predict the three-dimensional structures of proteins based on their amino acid sequences. This is crucial for understanding how proteins function and interact with other molecules.\n\n2. **Biological Research**: Researchers use AlphaFold to gain insights into various biological processes, including enzyme function, protein-protein interactions, and the mechanisms of diseases.\n\n3. **Drug Discovery**: By providing accurate protein structures, AlphaFold aids in the design of new drugs and therapeutic interventions by helping scientists understand the target proteins involved in diseases.\n\n4. **Genomics and Proteomics**: AlphaFold can assist in interpreting genomic data by predicting the structures of proteins encoded by newly sequenced genes, which can help in annotating genomes and u

#### Runnables

```
runnables = [prompt, llm, parser]
```

Runnables can be 
- executed
    - `runnable.invoke()`, 
    - `runnable.batch()` 
    - `runnable.stream()`
- inspected
    - `runnable.get_input_schema()`
    - `runnable.get_output_schema()`
    - `runnable.config_schema()`
- composed

In [ ]:
chain = RunnableSequence(prompt, llm, parser)

In [ ]:
chain.invoke({"tool": "Alphafold"})

'AlphaFold is an artificial intelligence program developed by DeepMind that predicts protein structures with high accuracy. It uses deep learning techniques to analyze the amino acid sequences of proteins and predict their three-dimensional shapes. Understanding protein structures is crucial for various fields, including:\n\n1. **Biochemistry and Molecular Biology**: AlphaFold helps researchers understand how proteins function, interact, and contribute to biological processes.\n\n2. **Drug Discovery**: By providing insights into protein structures, AlphaFold can aid in the design of new drugs that target specific proteins associated with diseases.\n\n3. **Genetics**: It can help interpret the effects of genetic mutations on protein structure and function, which is important for understanding genetic disorders.\n\n4. **Synthetic Biology**: Researchers can use AlphaFold to design new proteins with desired functions for applications in biotechnology.\n\n5. **Evolutionary Biology**: The pr

In [65]:
for chunk in chain.stream({"tool": "Alphafold"}):
    print(chunk, end="", flush=True)

AlphaFold is an artificial intelligence program developed by DeepMind that predicts protein structures with high accuracy. It is primarily used for:

1. **Protein Structure Prediction**: AlphaFold can predict the three-dimensional structures of proteins based on their amino acid sequences. This is crucial for understanding how proteins function and interact with other molecules.

2. **Biological Research**: Researchers use AlphaFold to gain insights into various biological processes, including enzyme function, protein-protein interactions, and the mechanisms of diseases.

3. **Drug Discovery**: By providing accurate protein structures, AlphaFold aids in the design of new drugs and therapeutic interventions by helping scientists understand the target proteins involved in diseases.

4. **Genomics and Proteomics**: AlphaFold can assist in interpreting genomic data by predicting the structures of proteins encoded by newly sequenced genes, which can help in annotating genomes and understand

In [ ]:
chain.batch(
    [
        {"tool": "Alphafold"},
        {"tool": "RFDiffusion"},
    ]
)

['AlphaFold is an artificial intelligence program developed by DeepMind that predicts protein structures with high accuracy. It is primarily used for:\n\n1. **Protein Structure Prediction**: AlphaFold can predict the three-dimensional structures of proteins based on their amino acid sequences. This is crucial for understanding how proteins function and interact with other molecules.\n\n2. **Biological Research**: Researchers use AlphaFold to gain insights into various biological processes, including enzyme function, protein-protein interactions, and the mechanisms of diseases.\n\n3. **Drug Discovery**: By providing accurate protein structures, AlphaFold aids in the design of new drugs and therapeutic interventions by helping scientists understand the target proteins involved in diseases.\n\n4. **Genomics and Proteomics**: AlphaFold can assist in interpreting genomic data by predicting the structures of proteins encoded by newly sequenced genes, which can help in annotating genomes and 

In [67]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


**Turn any function into a runnable**

In [ ]:
def double(x: int) -> int:
    return 2 * x


runnable = RunnableLambda(double)
runnable.invoke(2)

4

In [69]:
parallel_chain = RunnableParallel(
    double=RunnableLambda(lambda x: x * 2),
    triple=RunnableLambda(lambda x: x * 3),
)

parallel_chain.invoke(3)

{'double': 6, 'triple': 9}

In [70]:
parallel_chain.get_graph().print_ascii()

+------------------------------+   
| Parallel<double,triple>Input |   
+------------------------------+   
           **        **            
         **            **          
        *                *         
  +--------+          +--------+   
  | Lambda |          | Lambda |   
  +--------+          +--------+   
           **        **            
             **    **              
               *  *                
+-------------------------------+  
| Parallel<double,triple>Output |  
+-------------------------------+  


#### LCEL

In [72]:
chain = RunnableSequence(prompt, llm, parser)
chain

PromptTemplate(input_variables=['tool'], template='What is {tool} used for?')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x115689cc0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x11568b5b0>, root_client=<openai.OpenAI object at 0x11559e020>, root_async_client=<openai.AsyncOpenAI object at 0x115689c90>, model_name='gpt-4o-mini', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [73]:
prompt | llm | parser

PromptTemplate(input_variables=['tool'], template='What is {tool} used for?')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x115689cc0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x11568b5b0>, root_client=<openai.OpenAI object at 0x11559e020>, root_async_client=<openai.AsyncOpenAI object at 0x115689c90>, model_name='gpt-4o-mini', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [ ]:
chain.invoke({"tool": "Alphafold"})

'AlphaFold is an artificial intelligence program developed by DeepMind that predicts protein structures with high accuracy. It is primarily used for:\n\n1. **Protein Structure Prediction**: AlphaFold can predict the three-dimensional structures of proteins based on their amino acid sequences, which is a critical aspect of understanding biological functions.\n\n2. **Biological Research**: Researchers use AlphaFold to gain insights into protein functions, interactions, and mechanisms, which can aid in various fields such as biochemistry, molecular biology, and pharmacology.\n\n3. **Drug Discovery**: By providing accurate protein structures, AlphaFold can facilitate the identification of potential drug targets and the design of new therapeutics.\n\n4. **Understanding Diseases**: The program can help in studying the structural basis of diseases caused by protein misfolding or mutations, contributing to the development of treatments.\n\n5. **Synthetic Biology**: AlphaFold can assist in desi

### RAG

In [76]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

#### Loader

In [77]:
loader = ArxivLoader(
    query="llm reasoning with reinforcement learning",
    load_max_docs=1,
)
docs = loader.load()

In [78]:
len(docs)

1

In [79]:
len(docs[0].page_content)

46295

In [81]:
print(docs[0].page_content[:1000])

Advancing Reasoning in Large Language Models:
Promising Methods and Approaches
Avinash Patil
avinashpatil@ieee.org
ORCID: 0009-0002-6004-370X
Abstract—Large Language Models (LLMs) have succeeded
remarkably in various natural language processing (NLP) tasks,
yet their reasoning capabilities remain a fundamental challenge.
While LLMs exhibit impressive fluency and factual recall, their
ability to perform complex reasoning—spanning logical deduc-
tion, mathematical problem-solving, commonsense inference, and
multi-step reasoning—often falls short of human expectations.
This survey provides a comprehensive review of emerging tech-
niques enhancing reasoning in LLMs. We categorize existing
methods into key approaches, including prompting strategies
(e.g., Chain-of-Thought reasoning, Self-Consistency, and Tree-
of-Thought reasoning), architectural innovations (e.g., retrieval-
augmented models, modular reasoning networks, and neuro-
symbolic integration), and learning paradigms (e.g., fine-t

#### Splitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
all_splits = text_splitter.split_documents(docs)

print(f"Split Wikipedia page into {len(all_splits)} sub-documents.")

Split Wikipedia page into 177 sub-documents.


In [83]:
all_splits[0]

Document(metadata={'Published': '2025-02-05', 'Title': 'Advancing Reasoning in Large Language Models: Promising Methods and Approaches', 'Authors': 'Avinash Patil', 'Summary': 'Large Language Models (LLMs) have succeeded remarkably in various natural\nlanguage processing (NLP) tasks, yet their reasoning capabilities remain a\nfundamental challenge. While LLMs exhibit impressive fluency and factual\nrecall, their ability to perform complex reasoning-spanning logical deduction,\nmathematical problem-solving, commonsense inference, and multi-step\nreasoning-often falls short of human expectations. This survey provides a\ncomprehensive review of emerging techniques enhancing reasoning in LLMs. We\ncategorize existing methods into key approaches, including prompting strategies\n(e.g., Chain-of-Thought reasoning, Self-Consistency, and Tree-of-Thought\nreasoning), architectural innovations (e.g., retrieval-augmented models,\nmodular reasoning networks, and neuro-symbolic integration), and lea

#### Embeddings

In [84]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### Vector Store

In [85]:
vector_store = InMemoryVectorStore(embeddings)

document_ids = vector_store.add_documents(documents=all_splits)
document_ids[:10]

['b582217c-bbea-4a9e-af04-1a23c21b67ae',
 '611f6780-5a83-4f23-8ece-693f5540d7ee',
 'dd1562d1-c400-4ae3-ac30-17a27d695726',
 'e42682da-0d2e-42c6-91b4-f6ec41d131ae',
 'a81ec55d-1c69-45af-acb5-e466635cdf25',
 '622b7efd-ea9e-4880-b34c-295b992a86f7',
 '9867358f-fbb0-4907-bf9b-ad6842db7152',
 'f8f7e1fa-3e91-404f-8fc0-3e27e56d2041',
 '2e0a9213-ae1b-479c-9bea-c2fb1ccaa6ef',
 'f04eccea-ba74-4d55-9592-1f316d50ac42']

#### Retriever

In [86]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

#### Prompt

In [ ]:
template = ChatPromptTemplate(
    [
        ("system", "You are an assistant for question-answering tasks."),
        (
            "human",
            "Use the following pieces of retrieved context to answer the question. "
            "If you don't know the answer, just say that you don't know. "
            "Use three sentences maximum and keep the answer concise. "
            "\n# Question: \n-> {question} "
            "\n# Context: \n-> {context} "
            "\n# Answer: ",
        ),
    ]
)

In [ ]:
template.invoke({"context": "##CONTEXT##", "question": "##QUESTION##"}).to_messages()

[SystemMessage(content='You are an assistant for question-answering tasks.'),
 HumanMessage(content="Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. \n# Question: \n-> ##QUESTION## \n# Context: \n-> ##CONTEXT## \n# Answer: ")]

In [89]:
def format_docs(docs):
    formatted = "\n\n-> ".join(doc.page_content for doc in docs)
    return formatted

#### Generation

In [ ]:
question = "What is Program-Aided Language Models (PAL)?"
context = format_docs(retriever.invoke(question))
messages = template.invoke({"question": question, "context": context}).to_messages()
answer = llm.invoke(messages)

In [92]:
print(messages[1].content)

Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. 
# Question: 
-> What is Program-Aided Language Models (PAL)? 
# Context: 
-> selected based on a scoring or majority selection process
[14].
D. Program-aided Language Models (PAL)
Program-Aided Language Models (PAL) is a technique that
enhances a language model’s reasoning capabilities by allow-
ing it to call external computational tools—such as Python

-> arXiv:2409.12917, 2024.
[24] J. Wei et al., “Emergent abilities of large language models,” arXiv
preprint arXiv:2206.07682, 2022.
[25] L. Gao, A. Madaan, S. Zhou, U. Alon, P. Liu, Y. Yang, J. Callan,
and G. Neubig, “Pal: Program-aided language models,” in International

-> natural language understanding.
C. Reasoning in Large Language Models
Large Language Models (LLMs) such as GPT-4, PaLM, and
LLaMA utilize deep learning architectures, primarily t

In [93]:
answer

AIMessage(content="Program-Aided Language Models (PAL) is a technique that enhances a language model's reasoning capabilities by enabling it to call external computational tools, such as Python. This allows the model to perform more complex tasks and improve its overall performance in natural language understanding.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 304, 'total_tokens': 356, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'finish_reason': 'stop', 'logprobs': None}, id='run-a0850853-9739-435a-9db0-eead24c68fe6-0', usage_metadata={'input_tokens': 304, 'output_tokens': 52, 'total_tokens': 356})

#### RAG Chain

In [ ]:
rag_chain = (
    RunnableParallel(context=retriever | format_docs, question=RunnablePassthrough())
    | template
    | llm
)

In [95]:
rag_chain.invoke("What is Self-Consistency Prompting?")

AIMessage(content='Self-Consistency Prompting is an advanced technique that enhances reasoning accuracy by generating multiple diverse reasoning paths and selecting the most consistent one. It has been shown to significantly improve performance in structured domains like mathematics and logic. This method is part of a broader set of prompting strategies aimed at improving problem-solving and logical inference.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 257, 'total_tokens': 320, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'finish_reason': 'stop', 'logprobs': None}, id='run-889f2093-43e2-4374-bcfd-de3fa8ce1e5c-0', usage_metadata={'input_tokens': 257, 'output_tokens': 63, 'total_t

### Tools

In [97]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

#### Tool creation

In [98]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [99]:
tools = [multiply]
tool_map = {tool.name: tool for tool in tools}

#### Binding Tools

In [100]:
llm_with_tools = llm.bind_tools(tools)

In [101]:
question = "3 multiplied by 2"

In [ ]:
messages = [SystemMessage("You're a helpful assistant"), HumanMessage(question)]

In [103]:
ai_message = llm_with_tools.invoke(messages)

In [104]:
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pAZKCH0TCe2IGw0JVpIASM9n', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 54, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4761e33b-34c6-45c5-9423-0b3d0e31af0a-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_pAZKCH0TCe2IGw0JVpIASM9n', 'type': 'tool_call'}], usage_metadata={'input_tokens': 54, 'output_tokens': 18, 'total_tokens': 72})

In [105]:
messages.append(ai_message)

In [106]:
messages

[SystemMessage(content="You're a helpful assistant"),
 HumanMessage(content='3 multiplied by 2'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pAZKCH0TCe2IGw0JVpIASM9n', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 54, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4761e33b-34c6-45c5-9423-0b3d0e31af0a-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_pAZKCH0TCe2IGw0JVpIASM9n', 'type': 'tool_call'}], usage_metadata={'input_tokens': 54, 'output_tokens': 18, 'total_tokens': 72})]

#### Using Tool Calls

In [107]:
ai_message.additional_kwargs.get("tool_calls")

[{'id': 'call_pAZKCH0TCe2IGw0JVpIASM9n',
  'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'},
  'type': 'function'}]

In [ ]:
parsed_tool_calls = parse_tool_calls(ai_message.additional_kwargs.get("tool_calls"))

In [109]:
parsed_tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 2},
  'id': 'call_pAZKCH0TCe2IGw0JVpIASM9n',
  'type': 'tool_call'}]

In [ ]:
for tool_call in parsed_tool_calls:
    tool_call_id = tool_call["id"]

    function_name = tool_call["name"]
    arguments = tool_call["args"]

    func = tool_map[function_name]

    result = func.invoke(arguments)

    tool_message = ToolMessage(
        content=result,
        name=function_name,
        tool_call_id=tool_call_id,
    )
    messages.append(tool_message)

#### Sending the result back to the LLM

In [111]:
messages

[SystemMessage(content="You're a helpful assistant"),
 HumanMessage(content='3 multiplied by 2'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pAZKCH0TCe2IGw0JVpIASM9n', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 54, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4761e33b-34c6-45c5-9423-0b3d0e31af0a-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_pAZKCH0TCe2IGw0JVpIASM9n', 'type': 'tool_call'}], usage_metadata={'input_tokens': 54, 'output_tokens': 18, 'total_tokens': 72}),
 ToolMessage(con

In [112]:
ai_message = llm_with_tools.invoke(messages)

In [113]:
ai_message

AIMessage(content='3 multiplied by 2 is 6.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 79, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'finish_reason': 'stop', 'logprobs': None}, id='run-5883e596-8839-4121-a77a-b7be01977f22-0', usage_metadata={'input_tokens': 79, 'output_tokens': 11, 'total_tokens': 90})

### Agent

In [ ]:
load_dotenv()

True

In [116]:
class Agent:
    def __init__(
        self,
        name: str = "AI Agent",
        role: str = "Personal Assistant",
        instructions: str = "Help users with any question",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
        tools: List[StructuredTool] = [],
    ):

        self.name = name
        self.role = role
        self.instructions = instructions

        self.llm = ChatOpenAI(
            model=model,
            temperature=temperature,
        )

        self.tools = tools
        self.tool_map = {tool.name: tool for tool in tools}
        self.memory = [
            SystemMessage(
                content=f"You're {self.name}, your role is {self.role}, "
                f"and you need to {self.instructions} "
            ),
        ]

    def invoke(self, user_message: str):
        self.memory.append(HumanMessage(content=user_message))
        ai_message = self._invoke_llm()

        tool_calls = ai_message.additional_kwargs.get("tool_calls")
        if tool_calls:
            self._call_tools(tool_calls)
            self._invoke_llm()

        return self.memory[-1].content

    def _invoke_llm(self) -> AIMessage:
        llm = self.llm.bind_tools(self.tools)
        ai_message = llm.invoke(self.memory)
        self.memory.append(ai_message)
        return ai_message

    def _call_tools(self, tool_calls: List[Dict]):
        parsed_tool_calls = parse_tool_calls(tool_calls)
        for tool_call in parsed_tool_calls:
            tool_call_id = tool_call["id"]
            function_name = tool_call["name"]
            arguments = tool_call["args"]
            func = self.tool_map[function_name]
            result = func.invoke(arguments)
            tool_message = ToolMessage(
                content=result,
                name=function_name,
                tool_call_id=tool_call_id,
            )
            self.memory.append(tool_message)

In [117]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [118]:
agent = Agent(tools=[multiply])

In [119]:
agent.invoke("2 multiplied by 2")

'2 multiplied by 2 is 4.'

In [120]:
agent.memory

[SystemMessage(content="You're AI Agent, your role is Personal Assistant, and you need to Help users with any question "),
 HumanMessage(content='2 multiplied by 2'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_du6moZBacMTbkVtczl2XqRpW', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 69, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8c7a7aad-7679-41e0-a987-e2d35829a4d7-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 2}, 'id': 'call_du6moZBacMTbkVtczl2XqRpW', 'type': 'tool_call'}], usage_metadata={'input_tok